In [1]:
from keras import models
import pandas as pd 
import keras
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.engine import sequential 
from keras.models import Sequential
from keras.layers import Dense, Flatten
from numpy.random import seed
import os
import time
import tensorflow as tf
import pandas as pd
import keras.backend as K
from csv import writer
from csv import reader


In [2]:
data = pd.read_csv('.\Banknote_seed_result/banknote.csv')

In [3]:
seeds = list(range(0,10))
multiply= [0.5,1,2]
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [4]:
X, y = data.values[:, :-1], data.values[:, -1]

In [5]:
for mltp in multiply:
    for actual_seed in seeds:
        seed(actual_seed)
        tf.random.set_seed(actual_seed)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state= actual_seed)


        n_features = X.shape[1]
        model = Sequential()
        model.add(Dense(10*mltp, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
        model.add(Dense(1, activation='sigmoid'))

        filename = ".\Banknote_seed_result/"+"Banknote_" + str(mltp) + "_" +str(actual_seed)+ ".csv"
        history_logger= keras.callbacks.CSVLogger(filename, separator=",", append=True)
        time_callback = TimeHistory()
        model.compile(optimizer="adam", loss='binary_crossentropy')
        history = model.fit(X_train, y_train,
                            epochs=100, batch_size=32, verbose=0, 
                            validation_data=(X_test,y_test),
                            callbacks=[history_logger,time_callback])
        
        times = time_callback.times
        csv_input = pd.read_csv(filename)
        csv_input['time'] = times
        csv_input.to_csv(filename, index=False)
        K.clear_session()
        del model
        tf.compat.v1.reset_default_graph()